In [ ]:
%pyspark
from pyspark import SparkFiles
# Load apparel data from S3 into a DataFrame

sports_df = spark.read.option('header', 'true').csv("s3a://amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz", inferSchema=True, sep='\t')
sports_df.show(10)

In [ ]:
%pyspark
from pyspark.sql.functions import to_date

# review_date should be in the format yyyy-mm-dd
date_df = sports_df.withColumn("date", to_date("review_date", "yyyy-mm-dd"))
date_df.show(10)

In [ ]:
%pyspark
from pyspark.sql.functions import col

review_df = date_df.select(["review_id", "customer_id", "product_id", "product_parent", col("date").alias("review_date")])
review_df.show(10)

In [ ]:
%pyspark
# Number of reviews
review_df.count() 

In [ ]:
%pyspark

products_df = date_df.select(["product_id", "product_title"]).distinct()
products_df.show(10)

In [ ]:
%pyspark
# Number of products
products_df.count()

In [ ]:
%pyspark
# Get the products from the database
dbproducts_df = sqlContext.read.jdbc(url=jdbc_url, table='products', properties=config)
dbproducts_df.show(10)

In [ ]:
%pyspark
# Join the two products data frames (right excluding join to have only the products that are not in the database)
new_prod_df = products_df.join(dbproducts_df, on="product_id", how="leftanti")
new_prod_df.show(10)

In [ ]:
%pyspark
# Drop duplicates of the new data frame that came from apparel data set
new_prod_df = new_prod_df.select(['product_id', 'product_title']).dropDuplicates(['product_id'])
new_prod_df.count()

In [ ]:
%pyspark

customers_df = date_df.groupBy("customer_id").count().orderBy("customer_id")
customers_df.show(10)

In [ ]:
%pyspark
customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df.show(10)

In [ ]:
%pyspark
customers_df.count()

In [ ]:
%pyspark

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

vine_df = date_df.select(["review_id", col("star_rating").cast(IntegerType()), "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

In [ ]:
%pyspark
# Number of vines
vine_df.count()

In [ ]:
%pyspark
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:<port>/<db>"
config = {"user":"<usrname>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
%pyspark
# Append DataFrame to review_id_table in RDS
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
%pyspark
# Write dataframe to products table in RDS
new_prod_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
%pyspark
# Write dataframe to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
%pyspark

vine_df.write.jdbc(url=jdbc_url , table='vine_table', mode=mode, properties=config)

In [ ]:
%pyspark

vine_df.write.jdbc(url=jdbc_url , table='vine_table_sports', mode=mode, properties=config)